In [1]:
import numpy as np
from threading import Thread

In [17]:
A = np.genfromtxt("A.txt", delimiter=",")
b = np.genfromtxt("b.txt", delimiter=",")

In [18]:
A

array([[ 0.4974,  0.    , -0.1299,  0.0914,  0.1523],
       [-0.0305,  0.3248,  0.    , -0.0619,  0.0203],
       [ 0.0102, -0.0914,  0.5887,  0.0112,  0.0355],
       [ 0.0305,  0.    , -0.0741,  0.5887,  0.    ],
       [ 0.0203, -0.0305,  0.1472, -0.0122,  0.4263]])

In [19]:
b

array([ 1.5875, -1.759 ,  1.4139,  1.7702, -2.0767])

## Метод квадратного корня

In [2]:
def cholessky_decomposition(A):
    S = np.zeros_like(A)
    S[0,0] = np.sqrt(A[0,0])
    S[0,1:] = A[0,1:]/S[0,0]
    for i in range(1, S.shape[0]):
        S[i, i] = np.sqrt(np.abs(A[i, i] - np.sum(S[0:i, i]**2)))
        for j in range(i+1, S.shape[1]):
            S[i][j] = (A[i][j] - np.dot(S[0:i, i], S[0:i, j])) / S[i][i]
    return S

In [3]:
def back_substitution(A, b, start_from="bottom"):
    if(start_from == "top"):
        return substitute_from_top(A, b)
    elif(start_from == "bottom"):
        return substitute_from_bottom(A, b)

In [4]:
def substitute_from_top(A, b):
    x = np.zeros_like(b)
    for i in range(0, A.shape[0]):
        x[i] = (b[i] - np.dot(A[i, 0:i], x[0:i])) / A[i][i]
    return x

In [5]:
def substitute_from_bottom(A, b):
    x = np.zeros_like(b)
    n = A.shape[0] - 1
    zero = -1
    for i in range(n, zero, -1):
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i,i]
    return x

In [6]:
def solve(A, b):
    U = cholessky_decomposition(A)
    y = back_substitution(U.T, b, "top")
    x = back_substitution(U, y, "bottom")
    return x

In [20]:
%timeit solve(np.dot(A.T, A), b)

179 µs ± 1.47 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Метод встречной прогонки

In [21]:
np.zeros(np.int(np.array([1, 1, 1]).shape[0]/2))

array([0.])

In [22]:
a = TriagonalMethod(A, np.arange(0,1000))

NameError: name 'TriagonalMethod' is not defined

In [ ]:
%timeit a.solve()

In [ ]:
%timeit np.linalg.solve(A, np.arange(0,1000))

## Метод отражений

In [34]:
class HausholderMethod:
    def __init__(self, A, b):
        self.A = np.copy(A)
        self.b = np.copy(b)
        self.n = self.b.shape[0]
        self.iteration = 0
        
    def decompose(self):
        for k in range(0, self.n-1):
            self.iteration = k
            s = self.get_s()
            e = self.get_e()
            alpha = self.calculate_alpha(s)
            omega = self.calculate_omega(alpha, s, e)
            A_ = np.copy(self.A[k:,k+1:])
            b_ = np.copy(self.b[k:])
            self.A[k][k] = alpha
            for i in range(k, self.n):
                self.b[i] = b_[i-k] - 2*omega[i]*np.dot(b_, omega[k:])
                for j in range(k+1, self.n):
                    self.A[i][j] = self.A[i][j] - 2*omega[i]*np.dot(A_[:,j-(k+1)], omega[k:])
            self.A[k+1:,k] = 0
    
    def solve(self):
        self.decompose()
        return back_substitution(self.A, self.b)
            
    def get_s(self):
        s = np.copy(self.A[:,self.iteration])
        s[:self.iteration] = 0
        return s
    
    def get_e(self):
        e = np.zeros(self.n)
        e[self.iteration] = 1
        return e
        
    def calculate_alpha(self, s):
        return np.linalg.norm(s)
    
    def calculate_x(self, alpha, s, e):
        return 1 / (np.sqrt(2*np.dot(s, s - alpha * e)))
    
    def calculate_omega(self, alpha, s, e):
        x = self.calculate_x(alpha, s, e)
        return x * (s - alpha*e)

In [37]:
print("A = \n{}\n".format(A))
print("b = \n{}\n".format(b))
x = HausholderMethod(A, b).solve()
np.dot(A, x) - b

A = 
[[ 0.4974  0.     -0.1299  0.0914  0.1523]
 [-0.0305  0.3248  0.     -0.0619  0.0203]
 [ 0.0102 -0.0914  0.5887  0.0112  0.0355]
 [ 0.0305  0.     -0.0741  0.5887  0.    ]
 [ 0.0203 -0.0305  0.1472 -0.0122  0.4263]]

b = 
[ 1.5875 -1.759   1.4139  1.7702 -2.0767]



array([-2.22044605e-16,  0.00000000e+00,  0.00000000e+00,  2.22044605e-16,
        0.00000000e+00])

## Метод простых итераций

In [49]:
class IterativeMethod:
    def __init__(self, A, b, x0, eps=1e-8):
        self.B = 0
        self.g = 0
        self.x = x0
        self.canonize(A, b)
        self.eps = eps
        
    def canonize(self, A, b):
        self.B = np.eye(*A.shape) - np.dot(A.T, A)/np.linalg.norm(np.dot(A.T, A))
        self.g = np.dot(A.T, b) / np.linalg.norm(np.dot(A.T, A))
        

    def solve(self):
        difference = 1
        it = 0
        x_prev = float('inf')
        while(difference >= self.eps and it < 1000):
            x_prev = np.copy(self.x)
            self.x = np.dot(self.B, self.x) + self.g
            difference = np.linalg.norm(self.x - x_prev)
            print("="*50)
            print("Accuracy is {0}\nIteration is {1}".format(difference, it))
            it += 1
        return x_prev

In [50]:
x = IterativeMethod(A, b, np.random.rand(5)).solve()

Accuracy is 2.5326582024911923
Iteration is 0
Accuracy is 1.550768824053721
Iteration is 1
Accuracy is 1.1305204710476853
Iteration is 2
Accuracy is 0.8847930054577675
Iteration is 3
Accuracy is 0.7107559016620469
Iteration is 4
Accuracy is 0.5767852603655427
Iteration is 5
Accuracy is 0.4701064351141288
Iteration is 6
Accuracy is 0.3839336674946003
Iteration is 7
Accuracy is 0.31387435873664676
Iteration is 8
Accuracy is 0.2567398361591423
Iteration is 9
Accuracy is 0.2100739813185109
Iteration is 10
Accuracy is 0.17192809744192367
Iteration is 11
Accuracy is 0.14073302135235732
Iteration is 12
Accuracy is 0.11521568909973667
Iteration is 13
Accuracy is 0.09433927253282769
Iteration is 14
Accuracy is 0.07725764789014375
Iteration is 15
Accuracy is 0.06327959206532019
Iteration is 16
Accuracy is 0.05184010881052445
Iteration is 17
Accuracy is 0.04247723822313427
Iteration is 18
Accuracy is 0.03481320258618519
Iteration is 19
Accuracy is 0.028539035176621725
Iteration is 20
Accuracy is 

In [46]:
np.dot(A, x) - b

array([-5.25411914e-09, -1.59552580e-08, -6.48938370e-09, -1.05311160e-09,
        8.22889712e-09])

## Метод Гаусса-Зейделя

In [51]:
class GaussSeidelMethod:
    def solve(self, A, b, x0):
        x = np.copy(x0)
        

SyntaxError: unexpected EOF while parsing (<ipython-input-51-40e280b2ec39>, line 1)